In [ ]:
from filterpy.kalman import KalmanFilter

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
import pymc as pm
import arviz as az
from retry import retry
from tqdm import tqdm
from fastprogress import fastprogress
import plotly.graph_objects as go
fastprogress.printing = lambda: True

In [4]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats, playergamelog, boxscoretraditionalv3, boxscoreusagev3
from nba_api.stats.endpoints import commonplayerinfo
import pandas as pd
import time

**3 point percentage**

In [ ]:
@retry()
def get_player_game_percentages(player_id):
  all_seasons = playercareerstats.PlayerCareerStats(player_id=player_id).get_data_frames()[0]["SEASON_ID"].tolist()
  logs = []
  for i in all_seasons:
    logs.append(playergamelog.PlayerGameLog(player_id=player_id, season=i).get_data_frames()[0])
    time.sleep(.25)

  log = pd.concat(logs)
  log = log.sort_values('Game_ID')

  attempts = log["FG3A"]
  makes = log["FG3M"]
  total_attempts = attempts.sum()
  total_makes = makes.sum()
  career_percentage = total_makes / total_attempts

  return attempts.tolist(), makes.tolist(), career_percentage

In [ ]:
def convert_to_binary_array(attempts, makes):
    arr = []
    for attempt, make in zip(attempts, makes):
        arr.extend([1] * make)
        arr.extend([0] * (attempt - make))
    return arr

In [ ]:
def b3P(player_id):
  shooting_attempts, shooting_successes, perc = get_player_game_percentages(player_id)
  shots = convert_to_binary_array(shooting_attempts, shooting_successes)

  with pm.Model() as model:

      theta = pm.Beta('theta', alpha=1, beta=1)
      y = pm.Bernoulli('y', p=theta, observed=shots)

      start = pm.find_MAP()
      step = pm.Metropolis()
      trace = pm.sample(len(shots), step=step, start=start)
      summary_stats = az.summary(trace, var_names=['theta'], hdi_prob=0.95)
      mean = summary_stats.loc['theta', 'mean']
      hdi_lower = summary_stats.loc['theta', 'hdi_2.5%']
      hdi_upper = summary_stats.loc['theta', 'hdi_97.5%']

  #return mean_shooting_skill, hdi_lower, hdi_upper

  return trace, mean, hdi_lower, hdi_upper

In [ ]:
def b3P_arr(player_id, stepsize=None, progressbar=False):
  shooting_attempts, shooting_successes, perc = get_player_game_percentages(player_id)
  shots = convert_to_binary_array(shooting_attempts, shooting_successes)
  mean_arr = []
  hdi_lower_arr = []
  hdi_upper_arr = []

  if stepsize == None:
    stepsize = int(len(shots) / 100)
  for i in tqdm(range(0, len(shots), stepsize)):
    with pm.Model() as model:
      upper_ind = int(max(len(shots), i + stepsize))
      subset_shots = shots[0:upper_ind]
      theta = pm.Beta('theta', alpha=1, beta=1)
      y = pm.Bernoulli('y', p=theta, observed=subset_shots)

      start = pm.find_MAP()
      step = pm.Metropolis()
      trace = pm.sample(len(subset_shots), step=step, start=start, progressbar=progressbar)
      summary_stats = az.summary(trace, var_names=['theta'], hdi_prob=0.95)
      mean = summary_stats.loc['theta', 'mean']
      hdi_lower = summary_stats.loc['theta', 'hdi_2.5%']
      hdi_upper = summary_stats.loc['theta', 'hdi_97.5%']

    mean_arr.append(mean)
    hdi_lower_arr.append(hdi_lower)
    hdi_upper_arr.append(hdi_upper)

  return mean_arr, hdi_lower_arr, hdi_upper_arr

In [ ]:
def kalman(mean, burnin):
  kf = KalmanFilter(dim_x=1, dim_z=1)
  kf.F = np.array([[1.]])
  kf.H = np.array([[1.]])

  q_var = np.var(mean[:burnin])
  initial_r_var = q_var
  kf.Q = np.array([[q_var]])
  kf.R = np.array([[q_var]])

  x_rest = list()
  kf.x = np.array([0])
  kf.P *= 10.
  test = mean[burnin:]
  decay_rate = .01
  for i, y in enumerate(test[:-1]):
    kf.predict()

    #Weighting
    weight = np.exp(decay_rate * i)
    kf.R = np.array([[initial_r_var / weight]])

    x_rest.append(kf.x[0])
    kf.update([y])

  return [0] + list(mean[:burnin]) + x_rest



In [ ]:
def kBP(player_id, burnin=10, stepsize=1):
  mean, hdi_lower, hdi_upper = b3P_arr(player_id, stepsize=stepsize)
  xs = kalman(mean, burnin)
  return xs[-1]

In [ ]:
xs = kBP(1629636)

**Plus-minus**

In [13]:
@retry()
def get_season_game_logs(player_id, season_id):
  return playergamelog.PlayerGameLog(player_id=player_id, season=season_id).get_data_frames()[0]

In [15]:
@retry()
def get_margin(game_id, W_or_L):
  boxscore = boxscoretraditionalv3.BoxScoreTraditionalV3(game_id=game_id)
  boxscore = boxscore.get_data_frames()[2]
  unsigned_margin = np.absolute(boxscore["points"].iloc[1] - boxscore["points"].iloc[0])

  if W_or_L == "W":
    return unsigned_margin
  else:
    return -1 * unsigned_margin

In [16]:
@retry()
def get_usage_pct(game_id, player_id):
  usage_data = boxscoreusagev3.BoxScoreUsageV3(game_id=game_id).get_data_frames()[0]
  usage_pct = usage_data[usage_data["personId"] == player_id]["usagePercentage"].iloc[0]
  return usage_pct

In [17]:
@retry()
def get_all_seasons(player_id):
  return playercareerstats.PlayerCareerStats(player_id=player_id).get_data_frames()[0]["SEASON_ID"].tolist()

In [18]:
def get_game_pm(player_id):
  all_seasons = get_all_seasons(player_id)
  logs = []
  print("Collecting season game logs")
  for season in tqdm(all_seasons):
    logs.append(get_season_game_logs(player_id, season))
    time.sleep(.25)

  log = pd.concat(logs)
  log = log.sort_values('Game_ID')

  margin = []
  usg = []
  """
  print("Collecting margins and usage for each game")
  for index, row in log.iterrows():
    print("g" + str(index) + "/" + str(len(log)))
    margin.append(get_margin(row["Game_ID"], row["WL"]))
    #usg.append(get_usage_pct(row["Game_ID"], player_id))
    time.sleep(.25)


  log["MARGIN"] = margin
  #log["USG"] = usg
  """
  return log[["PLUS_MINUS", "MIN"]]


In [19]:
def normalize_list(input_list):
    min_val = min(input_list)
    max_val = max(input_list)
    normalized_list = [(val - min_val) / (max_val - min_val) for val in input_list]

    return normalized_list


In [20]:
def calculate_exponential_weight(index, coeff):
  return coeff * (1 - (1 / np.exp(index)))

In [21]:
def get_weighted_pm_stats(raw_df):
  REL_MIN = normalize_list(raw_df["MIN"].tolist())
  PM = raw_df["PLUS_MINUS"].tolist()
  WEIGHTED_PM = []
  for i in range(len(REL_MIN)):
    WEIGHTED_PM.append(
        calculate_exponential_weight(i, REL_MIN[i]) * PM[i]
    )

  return WEIGHTED_PM

In [22]:
def b_infer(weighted_pm_list):
  with pm.Model() as model:

      theta = pm.Beta('theta', alpha=1, beta=1)
      y = pm.Normal('y', mu=theta, sigma=np.std(weighted_pm_list), observed=weighted_pm_list)

      start = pm.find_MAP()
      step = pm.Metropolis()
      trace = pm.sample(len(weighted_pm_list), step=step, start=start)
      summary_stats = az.summary(trace, var_names=['theta'], hdi_prob=0.95)
      mean = summary_stats.loc['theta', 'mean']
      hdi_lower = summary_stats.loc['theta', 'hdi_2.5%']
      hdi_upper = summary_stats.loc['theta', 'hdi_97.5%']

      posterior_means = trace.posterior["theta"].to_numpy().tolist()


  return posterior_means, mean, hdi_lower, hdi_upper


In [23]:
def compute_bWPM(player_id):
  raw = get_game_pm(player_id)
  weighted_pm = get_weighted_pm_stats(raw)
  posterior_means, mean, hdi_lower, hdi_upper = b_infer(weighted_pm)
  return posterior_means, mean, hdi_lower, hdi_upper


In [25]:
active_players = pd.json_normalize(players.get_active_players())
players_top_mins_per_game = pd.read_csv("players_top_mins_per_game.csv")["TEAM"].tolist()
active_players = active_players[active_players["full_name"].isin(players_top_mins_per_game)]
name = active_players["full_name"].tolist()
ids = active_players["id"].tolist()

In [26]:
means = []
lower_bounds = []
upper_bounds = []
for i in tqdm(range(len(ids))):
  try:
    posterior_means, mean, hdi_lower, hdi_upper = compute_bWPM(ids[i])
    means.append(mean)
    lower_bounds.append(hdi_lower)
    upper_bounds.append(hdi_upper)
  except:
    continue

out = {
    "PLAYER_NAME": name,
    "PLAYER_ID": ids,
    "MEAN": means,
    "HDI_LOWER_BOUND": lower_bounds,
    "HDI_UPPER_BOUND": upper_bounds
}
out_df = pd.DataFrame(out)
out_df.to_csv("bwPM_results.csv")

  0%|          | 0/198 [00:00<?, ?it/s]


100%|██████████| 7/7 [00:03<00:00,  1.89it/s]


<ipython-input-22-1209440969e7>:9: FutureWarning: The `start` kwarg was renamed to `initvals` and can now do more. Please check the docstring.
  trace = pm.sample(len(weighted_pm_list), step=step, start=start)


/usr/local/lib/python3.10/dist-packages/arviz/utils.py:184: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  numba_fn = numba.jit(**self.kwargs)(self.function)
  1%|          | 1/198 [00:07<26:02,  7.93s/it]


100%|██████████| 3/3 [00:01<00:00,  1.91it/s]


...

<ipython-input-22-1209440969e7>:9: FutureWarning: The `start` kwarg was renamed to `initvals` and can now do more. Please check the docstring.
  trace = pm.sample(len(weighted_pm_list), step=step, start=start)


 99%|█████████▉| 197/198 [33:54<00:10, 10.73s/it]


100%|██████████| 10/10 [00:06<00:00,  1.66it/s]


<ipython-input-22-1209440969e7>:9: FutureWarning: The `start` kwarg was renamed to `initvals` and can now do more. Please check the docstring.
  trace = pm.sample(len(weighted_pm_list), step=step, start=start)


100%|██████████| 198/198 [34:07<00:00, 10.34s/it]


In [27]:
from scipy import stats
rel_bwpm = stats.zscore(out_df["MEAN"])

In [28]:
rel_bwpm

0      0.922831
1     -0.385400
2     -1.200203
3      0.844921
4     -0.333460
         ...   
193   -1.034646
194   -0.443832
195    0.520299
196   -0.664575
197    0.767012
Name: MEAN, Length: 198, dtype: float64

In [29]:
out_df["Z_SCORE"] = rel_bwpm

In [38]:
percentile = [int(stats.percentileofscore(out_df["MEAN"], i)) for i in out_df["MEAN"].tolist()]


In [39]:
len(percentile)

198

In [41]:
out_df["PERCENTILE"] = percentile

In [42]:
out_df.to_csv("bWPM.csv")